# Designing and Creating a Database

In the following project we will be designing and creating a databse to store historical data for games played in the Major Baseball League (MLB). The data has been pulled from the [Retrosheet](https://retrosheet.org/) website.

In [1]:
import pandas as pd
pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)

game_log = pd.read_csv('game_log.csv')
park_codes = pd.read_csv('park_codes.csv')
person_codes = pd.read_csv('person_codes.csv')
team_codes = pd.read_csv('team_codes.csv')

/dataquest/system/env/python3/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (12,13,14,15,19,20,81,82,83,84,85,86,87,88,93,94,95,96,97,98,99,100,105,106,108,109,111,112,114,115,117,118,120,121,123,124,126,127,129,130,132,133,135,136,138,139,141,142,144,145,147,148,150,151,153,154,156,157,160) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
print(game_log.shape)
print(game_log.head())
print(game_log.tail())

(171907, 161)
       date  number_of_game day_of_week v_name v_league  v_game_number h_name  \
0  18710504               0         Thu    CL1      NaN              1    FW1   
1  18710505               0         Fri    BS1      NaN              1    WS3   
2  18710506               0         Sat    CL1      NaN              2    RC1   
3  18710508               0         Mon    CL1      NaN              3    CH1   
4  18710509               0         Tue    BS1      NaN              2    TRO   

  h_league  h_game_number  v_score  h_score  length_outs day_night completion  \
0      NaN              1        0        2         54.0         D        NaN   
1      NaN              1       20       18         54.0         D        NaN   
2      NaN              1       12        4         54.0         D        NaN   
3      NaN              1       12       14         54.0         D        NaN   
4      NaN              1        9        5         54.0         D        NaN   

  forefeit p

The game log csv file is enormous, containing information on 171,907 games played with 161 columns of information including the date, team, score, location, umpires amongst others. It contains multiple id columns on which we may be able to link it to other files.

In [3]:
print(park_codes.shape)
print(park_codes.head())
print(park_codes.tail())

(252, 9)
  park_id                           name  \
0   ALB01                 Riverside Park   
1   ALT01                  Columbia Park   
2   ANA01       Angel Stadium of Anaheim   
3   ARL01              Arlington Stadium   
4   ARL02  Rangers Ballpark in Arlington   

                                        aka       city state       start  \
0                                       NaN     Albany    NY  09/11/1880   
1                                       NaN    Altoona    PA  04/30/1884   
2             Edison Field; Anaheim Stadium    Anaheim    CA  04/19/1966   
3                                       NaN  Arlington    TX  04/21/1972   
4  The Ballpark in Arlington; Ameriquest Fl  Arlington    TX  04/11/1994   

          end league                                           notes  
0  05/30/1882     NL  TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882  
1  05/31/1884     UA                                             NaN  
2         NaN     AL                                    

The park codes csv file contains the key information on the 252 baseball stadiums that games were played in. Notably it contains the city, state, league and id of the stadium, the last of which can be used to connect entries from the park codes dataset to that found in games logs.

In [4]:
print(person_codes.shape)
print(person_codes.head())
print(person_codes.tail())

(20494, 7)
         id     last   first player_debut mgr_debut coach_debut ump_debut
0  aardd001  Aardsma   David   04/06/2004       NaN         NaN       NaN
1  aaroh101    Aaron    Hank   04/13/1954       NaN         NaN       NaN
2  aarot101    Aaron  Tommie   04/10/1962       NaN  04/06/1979       NaN
3  aased001     Aase     Don   07/26/1977       NaN         NaN       NaN
4  abada001     Abad    Andy   09/10/2001       NaN         NaN       NaN
             id      last   first player_debut mgr_debut coach_debut ump_debut
20489  zuvep001   Zuvella    Paul   09/04/1982       NaN  04/02/1996       NaN
20490  zuveg101  Zuverink  George   04/21/1951       NaN         NaN       NaN
20491  zwild101  Zwilling   Dutch   08/14/1910       NaN  04/15/1941       NaN
20492  zycht001      Zych    Tony   09/04/2015       NaN         NaN       NaN
20493  thoma102  Thompson     NaN          NaN       NaN         NaN       NaN


The person codes csv file contains information on the 20,494 personel in various baseball leagues. It contains their full names, their debut date as a player, manager, coach or umpire. Additionally, it contains a unique id that can be linked to the main games logs file.

In [5]:
print(team_codes.shape)
print(team_codes.head())
print(team_codes.tail())

(150, 8)
  team_id league  start   end       city         nickname franch_id  seq
0     ALT     UA   1884  1884    Altoona  Mountain Cities       ALT    1
1     ARI     NL   1998     0    Arizona     Diamondbacks       ARI    1
2     BFN     NL   1879  1885    Buffalo           Bisons       BFN    1
3     BFP     PL   1890  1890    Buffalo           Bisons       BFP    1
4     BL1    NaN   1872  1874  Baltimore         Canaries       BL1    1
    team_id league  start   end        city   nickname franch_id  seq
145     WS8     NL   1886  1889  Washington   Senators       WS8    1
146     WS9     AA   1891  1891  Washington   Senators       WS9    1
147     WSN     NL   1892  1899  Washington   Senators       WS9    2
148     WSU     UA   1884  1884  Washington  Nationals       WSU    1
149     MIA     NL   2012     0       Miami    Marlins       FLO    2


The team codes csv contains basic information on 150 teams across multiple leagues. Thisinformation includes the teams inception year, year of termination, city, franchise association and a team id that can be used to identify it in the games logs dataset.

In [6]:
import sqlite3
conn = sqlite3.connect('mlb.db')

In [7]:
# Create the run_query and run_command functions that we previously used

DB = "mlb.db"

def run_query(q):
    with sqlite3.connect(DB) as conn:
        return pd.read_sql(q,conn)

def run_command(c):
    with sqlite3.connect(DB) as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(c)

def show_tables():
    q = '''
    SELECT
        name,
        type
    FROM sqlite_master
    WHERE type IN ("table","view");
    '''
    return run_query(q)


In [8]:
# Convert all of our DataFrames to tables in our database
game_log.to_sql('game_log', conn, index = False)
park_codes.to_sql('park_codes', conn, index = False)
person_codes.to_sql('person_codes', conn, index = False)
team_codes.to_sql('team_codes', conn, index = False)

In [9]:
# Add the new id column that we shall use to uniquely identify games
q = '''
ALTER TABLE game_log 
ADD COLUMN game_id TEXT
'''

run_command(q)

In [10]:
# Verify that we created the extra column
q = '''
SELECT * from game_log
LIMIT 5
'''

game_log_head = pd.read_sql_query(q, conn)
print(game_log_head)

       date  number_of_game day_of_week v_name v_league  v_game_number h_name  \
0  18710504               0         Thu    CL1     None              1    FW1   
1  18710505               0         Fri    BS1     None              1    WS3   
2  18710506               0         Sat    CL1     None              2    RC1   
3  18710508               0         Mon    CL1     None              3    CH1   
4  18710509               0         Tue    BS1     None              2    TRO   

  h_league  h_game_number  v_score  h_score  length_outs day_night completion  \
0     None              1        0        2         54.0         D       None   
1     None              1       20       18         54.0         D       None   
2     None              1       12        4         54.0         D       None   
3     None              1       12       14         54.0         D       None   
4     None              1        9        5         54.0         D       None   

  forefeit protest park_id

In [11]:
# Update all rows of the id column with the compound key desired
q = '''
UPDATE game_log SET game_id = h_name || date || number_of_game
'''

run_command(q)

# Check that the rows have been properly updated
q = '''
SELECT * FROM game_log
LIMIT 1
'''

row = pd.read_sql_query(q, conn)

# Normalizing the Data

We are going to look for ways to normalize our data and thus make our entire process simpler. There are a few things that we may want to look into when considering ways to normalize our data. These include:

   1. Repetition in columns: we do not want to be repeating the same type of columns over and over again
   2. Non-primary key columns should be attributes of the primary key: each table should contain information related to the primary key; otherwise we will have unrelated information in the same table
   3. Redundant data: storing the same information multiple times is wasteful and costly.

With these three points in mind, we turn to our database to see how we can apply these principles. Repetition in columns is most evident in the game_log table. Here we have multiple examples of where there are multiple columns describing one player followed by an equal number of columns describing the next player. Furthermore, this also violates the principle that the non-primary key columns should be attributes of the primary key. Since the game_log table should be focused on the game statistics and not information about the players, these columns should be placed into a new table and then removed from the game_log table. 

As for redundant data, there are a few examples. A simple one is the start and end dates for the park_codes table; the first and last occurence of a park code will be recorded in the game_log and hence these dates can be deduced from the game_log table. A similar principle can be applied to the start and end columns of the team_codes table, as a team's premiere and final game can be determined by the games_log table. 

A normalized database schema can be seen below. The code that follows will be creating the necessary tables and extracting relevant information from the originally imported data.

![DB Schema](mlb_schema.svg)

In [12]:
# Start creating the tables with no foreign keys
# 3 step process:
# 1. Create the new table
# 2. Insert the relevant information from the existing tables
# 3. Remove the original tables once we're done

cp1 = '''
CREATE TABLE IF NOT EXISTS person (
person_id TEXT PRIMARY KEY,
first_name TEXT,
last_name TEXT
);
'''

cp2 = '''
INSERT OR IGNORE INTO person
SELECT 
    id,
    first,
    last 
FROM person_codes;
'''

qp1 = '''
SELECT * from person
LIMIT 5
'''

run_command(cp1)
run_command(cp2)
pd.read_sql_query(qp1, conn)

,person_id,first_name,last_name
0,aardd001,David,Aardsma
1,aaroh101,Hank,Aaron
2,aarot101,Tommie,Aaron
3,aased001,Don,Aase
4,abada001,Andy,Abad


In [13]:
cp1 = '''
CREATE TABLE IF NOT EXISTS park(
park_id TEXT PRIMARY KEY,
name TEXT,
nickname TEXT,
city TEXT, 
state TEXT,
notes TEXT
);
'''

cp2 = '''
INSERT OR IGNORE INTO park
SELECT 
    park_id,
    name, 
    aka, 
    city, 
    state, 
    notes 
FROM park_codes;
'''

qp1 = '''
SELECT * FROM park
LIMIT 5
'''

run_command(cp1)
run_command(cp2)
pd.read_sql_query(qp1, conn)

,park_id,name,nickname,city,state,notes
0,ALB01,Riverside Park,None,Albany,NY,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,None,Altoona,PA,None
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,None
3,ARL01,Arlington Stadium,None,Arlington,TX,None
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,None


In [32]:
c1 = "DROP TABLE IF EXISTS appearance_type;"

run_command(c1)

c2 = """
CREATE TABLE appearance_type (
    appearance_type_id TEXT PRIMARY KEY,
    name TEXT,
    category TEXT
);
"""
run_command(c2)

appearance_type = pd.read_csv('appearance_type.csv')

with sqlite3.connect('mlb.db') as conn:
    appearance_type.to_sql('appearance_type',
                           conn,
                           index=False,
                           if_exists='append')

q = """
SELECT * FROM appearance_type;
"""

run_query(q)

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense
5,O6,Batter 6,offense
6,O7,Batter 7,offense
7,O8,Batter 8,offense
8,O9,Batter 9,offense
9,D1,Pitcher,defense


In [15]:
cl1 = '''
CREATE TABLE IF NOT EXISTS league (
    league_id TEXT PRIMARY KEY,
    name TEXT
);
'''

cl2 = '''
INSERT OR IGNORE INTO league
VALUES
    ("NL", "National League"),
    ("AL", "American League"),
    ("AA", "American Association"),
    ("FL", "Federal League"),
    ("PL", "Players League"),
    ("UA", "Union Association")
;
'''

ql1 = '''
SELECT * FROM league
'''

run_command(cl1)
run_command(cl2)
pd.read_sql_query(ql1,conn)

,league_id,name
0,NL,National League
1,AL,American League
2,AA,American Association
3,FL,Federal League
4,PL,Players League
5,UA,Union Association


In [16]:
ct3 = ''' 
DROP TABLE IF EXISTS team
'''
run_command(ct3)

In [17]:
# Create the team and game tables
ct1 = '''
CREATE TABLE IF NOT EXISTS team(
    team_id TEXT PRIMARY KEY,
    league_id TEXT,
    city TEXT,
    nickname TEXT,
    franch_id TEXT,
    FOREIGN KEY (league_id) REFERENCES league(league_id)
    );
'''

ct2 = '''
INSERT OR IGNORE INTO team
SELECT 
    team_id, 
    league,
    city,
    nickname,
    franch_id
FROM team_codes;
'''

qt1 = '''
SELECT * FROM team
LIMIT 5
'''

run_command(ct1)
run_command(ct2)
pd.read_sql_query(qt1, conn)

,team_id,league_id,city,nickname,franch_id
0,ALT,UA,Altoona,Mountain Cities,ALT
1,ARI,NL,Arizona,Diamondbacks,ARI
2,BFN,NL,Buffalo,Bisons,BFN
3,BFP,PL,Buffalo,Bisons,BFP
4,BL1,None,Baltimore,Canaries,BL1


In [18]:
# Create the game table

cg1 = '''
CREATE TABLE IF NOT EXISTS game(
    game_id TEXT PRIMARY KEY,
    date TEXT, 
    number_of_game INTEGER,
    park_id TEXT,
    length_outs INTEGER,
    day BOOLEAN,
    completion TEXT,
    forfeit TEXT,
    protest TEXT,
    attendance INTEGER,
    length_minutes INTEGER,
    additional_info TEXT,
    acquisition_info TEXT,
    FOREIGN KEY (park_id) REFERENCES park(park_id)
    );
'''

cg2 = '''
INSERT OR IGNORE INTO game
SELECT
    game_id,
    date,
    number_of_game,
    park_id,
    length_outs,
    CASE
        WHEN day_night = 'D' THEN 1
        WHEN day_night = 'N' THEN 0
        ELSE NULL
        END
        AS day,
    completion,
    forefeit,
    protest,
    attendance,
    length_minutes,
    additional_info,
    acquisition_info
FROM game_log;
'''

qg1 = '''
SELECT * FROM game
LIMIT 5
'''

run_command(cg1)
run_command(cg2)
pd.read_sql_query(qg1, conn)

,game_id,date,number_of_game,park_id,length_outs,day,completion,forfeit,protest,attendance,length_minutes,additional_info,acquisition_info
0,FW1187105040,18710504,0,FOR01,54,1,None,None,None,200,120,None,Y
1,WS3187105050,18710505,0,WAS01,54,1,None,None,None,5000,145,HTBF,Y
2,RC1187105060,18710506,0,RCK01,54,1,None,None,None,1000,140,None,Y
3,CH1187105080,18710508,0,CHI01,54,1,None,None,None,5000,150,None,Y
4,TRO187105090,18710509,0,TRO01,54,1,None,None,None,3250,145,HTBF,Y


In [19]:
# !rm mlb.db # command to remove all database files

In [20]:
ccl = '''
DROP TABLE IF EXISTS team_appearance;
'''

run_command(ccl)

cta1 = '''
CREATE TABLE IF NOT EXISTS team_appearance (
    team_id TEXT,
    game_id TEXT, 
    home BOOLEAN,
    league_id TEXT,
    score INTEGER,
    line_score TEXT,
    at_bats INTEGER,
    hits INTEGER,
    doubles INTEGER,
    triples INTEGER,
    homeruns INTEGER,
    rbi INTEGER,
    sacrifice_hits INTEGER,
    sacrifice_flies INTEGER,
    hit_by_pitch INTEGER,
    walks INTEGER,
    intentional_walks INTEGER,
    strikeouts INTEGER,
    stolen_bases INTEGER,
    caught_stealing INTEGER,
    grounded_into_double INTEGER,
    first_catcher_interference INTEGER,
    left_on_base INTEGER,
    pitchers_used INTEGER,
    individual_earned_runs INTEGER,
    team_earned_runs INTEGER,
    wild_pitches INTEGER,
    balks INTEGER,
    putouts INTEGER,
    assists INTEGER,
    errors INTEGER,
    passed_balls INTEGER,
    double_plays INTEGER,
    triple_plays INTEGER,
    PRIMARY KEY (team_id, game_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (league_id) REFERENCES league(league_id)
);

'''

run_command(cta1)

cta2 = '''
INSERT OR IGNORE INTO team_appearance
    SELECT
        h_name,
        game_id, 
        1 AS home,
        h_league,
        h_score,
        h_line_score,
        h_at_bats,
        h_hits,
        h_doubles,
        h_triples,
        h_homeruns,
        h_rbi,
        h_sacrifice_hits,
        h_sacrifice_flies,
        h_hit_by_pitch,
        h_walks,
        h_intentional_walks,
        h_strikeouts,
        h_stolen_bases,
        h_caught_stealing,
        h_grounded_into_double,
        h_first_catcher_interference,
        h_left_on_base,
        h_pitchers_used,
        h_individual_earned_runs,
        h_team_earned_runs,
        h_wild_pitches,
        h_balks,
        h_putouts,
        h_assists,
        h_errors,
        h_passed_balls,
        h_double_plays,
        h_triple_plays 
    FROM game_log
    
UNION

    SELECT
        v_name,
        game_id,
        0 AS home,
        v_league,
        v_score,
        v_line_score,
        v_at_bats,
        v_hits,
        v_doubles,
        v_triples,
        v_homeruns,
        v_rbi,
        v_sacrifice_hits,
        v_sacrifice_flies,
        v_hit_by_pitch,
        v_walks,
        v_intentional_walks,
        v_strikeouts,
        v_stolen_bases,
        v_caught_stealing,
        v_grounded_into_double,
        v_first_catcher_interference,
        v_left_on_base,
        v_pitchers_used,
        v_individual_earned_runs,
        v_team_earned_runs,
        v_wild_pitches,
        v_balks,
        v_putouts,
        v_assists,
        v_errors,
        v_passed_balls,
        v_double_plays,
        v_triple_plays
    FROM game_log;

'''
    

run_command(cta2)

qta = '''
SELECT * FROM team_appearance
LIMIT 5;
'''

pd.read_sql_query(qta, conn)

,team_id,game_id,home,league_id,score,line_score,at_bats,hits,doubles,triples,homeruns,rbi,sacrifice_hits,sacrifice_flies,hit_by_pitch,walks,intentional_walks,strikeouts,stolen_bases,caught_stealing,grounded_into_double,first_catcher_interference,left_on_base,pitchers_used,individual_earned_runs,team_earned_runs,wild_pitches,balks,putouts,assists,errors,passed_balls,double_plays,triple_plays
0,ALT,ALT188404300,1,UA,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,ALT,ALT188405020,1,UA,3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,ALT,ALT188405030,1,UA,5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,ALT,ALT188405050,1,UA,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,ALT,ALT188405100,1,UA,9,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [33]:
ccl = '''
DROP TABLE IF EXISTS person_appearance;
'''

run_command(ccl)

cpa1 = '''
CREATE TABLE IF NOT EXISTS person_appearance (
    appearance_id INTEGER PRIMARY KEY,
    person_id TEXT, 
    team_id TEXT,
    game_id TEXT,
    appearance_type_id,
    FOREIGN KEY (person_id) REFERENCES person(person_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (appearance_type_id) REFERENCES appearance_type(appearance_type_id)
);
'''

run_command(cpa1)

In [ ]:
template = """
INSERT INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "O{num}" 
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL

UNION

    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "D" || CAST({hv}_player_{num}_def_pos AS INT) 
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL;
"""


for hv in ["h","v"]:
    for num in range(1,10):
        query_vars = {
            "hv": hv,
            "num": num
        }
        # run commmand is a helper function which runs
        # a query against our database.
        run_command(template.format(**query_vars))

In [28]:
show_tables()

,name,type
0,game_log,table
1,park_codes,table
2,person_codes,table
3,team_codes,table
4,person,table
5,park,table
6,appearance_type,table
7,league,table
8,team,table
9,game,table
